In [3]:
import torch, torchvision
import matplotlib.pyplot as plt

In [5]:
training_transforms = torchvision.transforms.Compose([torchvision.transforms.RandomHorizontalFlip(p = 0.5), 
                                                      torchvision.transforms.Resize(size = (100, 100)), 
                                                      torchvision.transforms.ToTensor()])
validation_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize(size = (100, 100)), 
                                                        torchvision.transforms.ToTensor()])
test_transforms = torchvision.transforms.Compose([torchvision.transforms.Resize(size = (100, 100)), 
                                                  torchvision.transforms.ToTensor()])

training_dataset = torchvision.datasets.ImageFolder(root = "TRAINING SET PATH", transform = training_transforms)
validation_dataset = torchvision.datasets.ImageFolder(root = "VALIDATION SET PATH", transform = validation_transforms)
test_dataset = torchvision.datasets.ImageFolder(root = "TEST SET PATH", transform = test_transforms)

In [6]:
batch_size = 32
training_dataloader   = torch.utils.data.DataLoader(training_dataset, batch_size=batch_size, shuffle=True)
validation_dataloader = torch.utils.data.DataLoader(validation_dataset, batch_size=batch_size, shuffle=False)
test_dataloader       = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
class ConvNet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = torch.nn.Conv2d(in_channels=3, out_channels=16, kernel_size=5, stride=1)
        self.maxpool1 = torch.nn.MaxPool2d(kernel_size=3, stride=3)
        self.batchnorm1 = torch.nn.BatchNorm2d(num_features=16)

        self.conv2 = torch.nn.Conv2d(in_channels=16, out_channels=32, kernel_size=5, stride=1)
        self.maxpool2 = torch.nn.MaxPool2d(kernel_size=3, stride=3)
        self.batchnorm2 = torch.nn.BatchNorm2d(num_features=32)

        self.conv3 = torch.nn.Conv2d(in_channels=32, out_channels=32, kernel_size=5, stride=1)
        self.maxpool3 = torch.nn.MaxPool2d(kernel_size=3, stride=3)
        self.batchnorm3 = torch.nn.BatchNorm2d(num_features=32)

        self.flatten = torch.nn.Flatten()
        self.fc1 = torch.nn.Linear(32, 16)
        self.dropout = torch.nn.Dropout(0.1)
        self.fc2 = torch.nn.Linear(16, 2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.maxpool1(x)
        x = self.batchnorm1(x)
        x = torch.nn.functional.relu(x)
        
        x = self.conv2(x)
        x = self.maxpool2(x)
        x = self.batchnorm2(x)
        x = torch.nn.functional.relu(x)
        
        x = self.conv3(x)
        x = self.maxpool3(x)
        x = self.batchnorm3(x)
        x = torch.nn.functional.relu(x)

        x = self.flatten(x)
        
        x = self.fc1(x)
        x = torch.nn.functional.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        
        return x

In [ ]:
convnet = ConvNet().to('cuda')
test_input = torch.randn(16, 3, 100, 100, device='cuda')
test_output = convnet(test_input)
print(test_output.shape)

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(convnet.parameters())

In [9]:
def check_accuracy(neuralnet, dataloader):
    
    neuralnet.eval()
    
    true_prediction = 0
    total_prediction = 0

    device = next(neuralnet.parameters()).device
    
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = neuralnet(images)
            _, predicted = torch.max(outputs, 1)
            total_prediction += labels.size(0)
            true_prediction += (predicted == labels).sum().item()

    accuracy = true_prediction / total_prediction
    
    return accuracy    

In [ ]:
test_accuracy = check_accuracy(convnet, test_dataloader)
print(f'Test accuracy before training: {test_accuracy * 100 :.2f}%')

In [11]:
def train_one_epoch(neuralnet, dataloader, optimizer, loss_fn):

    total_loss = 0.0 
    
    neuralnet.train()
    
    for imgs_batch, labels_batch in dataloader:
        imgs_batch, labels_batch = imgs_batch.to("cuda"), labels_batch.to("cuda")
        optimizer.zero_grad()
        outputs = neuralnet(imgs_batch)
        loss = loss_fn(outputs, labels_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()

    average_loss = total_loss / len(dataloader)

    return average_loss

In [ ]:
training_accuracy_list = []
validation_accuracy_list = []

NUM_EPOCHS = 10
scheduling = 2
earlystop = 5
best_val_accuracy = 0.0
no_improvement = 0

for epoch_no in range(NUM_EPOCHS):
    print(f'Epoch {epoch_no + 1}...')
    
    train_one_epoch(convnet, training_dataloader, optimizer, loss_fn)
    
    training_accuracy = check_accuracy(convnet, training_dataloader)
    print(f'Training accuracy: {training_accuracy * 100 :.2f}%')
    validation_accuracy = check_accuracy(convnet, validation_dataloader)
    print(f'Validation accuracy: {validation_accuracy * 100 :.2f}%')
    
    training_accuracy_list.append(training_accuracy)
    validation_accuracy_list.append(validation_accuracy)

    if validation_accuracy > best_val_accuracy:
        best_val_accuracy = validation_accuracy
        no_improvement = 0
    else:
        no_improvement += 1

    if no_improvement >= scheduling:
        for param_group in optimizer.param_groups:
            param_group['lr'] *= 0.1
        print(f'Learning rate reduced to {param_group["lr"]}')
    
    if no_improvement >= earlystop:
        print(f'No improvement for {earlystop} epochs.')
        break
    

In [ ]:
plt.plot(training_accuracy_list)
plt.plot(validation_accuracy_list)
plt.xlabel("Epochs")
plt.ylabel("Accuracy")

In [ ]:
test_accuracy = check_accuracy(convnet, test_dataloader)
print(f'Test accuracy after training: {test_accuracy * 100 :.2f}%')

In [15]:
torch.save(convnet.state_dict(), "convNet_for_imageClassification.pth")